In [ ]:
from Analizador import AnalisisTexto
import pymysql
import sys
import spacy
from DBdata2 import *

In [ ]:
class Datos:
    con= None;
    global analisis 
    global data
    data = Datos2()
    analisis= AnalisisTexto()

In [ ]:
def conexionDB():
    try:
        conexion = pymysql.connect(host='localhost', user='root', password='', db='data')
    except (pymysql.err.OperationalError, pymysql.err.InternalError) as e:
        print("Ocurrió un error al conectar: ", e)
    return conexion

In [ ]:
def guardarTripletas(id,s,p,o):
    db = pymysql.connect(host='localhost', user='root', password='', db='data')
    cur = db.cursor()
    sql = """INSERT INTO tripletas(Id ,sujeto, predicado, objeto) VALUES (%s, %s, %s, %s) """
    datos= (id,s,p,o)
    cur.execute(sql, datos)
    db.commit()
    print ("tripletas guardadas....")
    cur.close()

In [ ]:
def corregirResumen():
    db = pymysql.connect(host='localhost', user='root', password='', db='data')
    cur = db.cursor()
    cur.execute (u""" UPDATE tripletas
SET objeto = trim(substr(objeto, 1, INSTR(objeto, 'Keywords:')-1))
WHERE predicado = 'https://w3id.org/scholarlydata/ontology/conference-ontology.owl#abstract'
AND objeto LIKE '%Keywords:%'; """)
    db.commit()
    cur.close()

In [ ]:
def extraerDatos():
    db = pymysql.connect(host='localhost', user='root', password='', db='data')
    cur = db.cursor()
    cur.execute(u"""SELECT DISTINCT sujeto, group_concat(IF(predicado = 'https://w3id.org/scholarlydata/ontology/conference-ontology.owl#title',
        objeto, '')) titulo, group_concat(IF(predicado = 'https://w3id.org/scholarlydata/ontology/conference-ontology.owl#abstract', objeto, ''))
        resumen FROM tripletas WHERE predicado = 'https://w3id.org/scholarlydata/ontology/conference-ontology.owl#abstract' OR  predicado = 'https://w3id.org/scholarlydata/ontology/conference-ontology.owl#title' GROUP BY sujeto;""")
    filas = cur.fetchall()
        for fila in filas:
            paper = fila[0]
            titulo = fila[1].strip(',')
            resumen = fila[2].strip(',')
            listAbs = resumen.split('.')
            print(titulo,"lista")
            pos = 1
            for frase in listAbs:
                longitud=len(frase.split(' '))
                if longitud > 3:
                    entidad= analisis.get_entities(frase)
                    relacion=analisis.get_relacion(frase)
                    print (40 * "*", frase)
                    print("Sujeto ->", entidad[0], "Predicado ->", relacion, "Objeto ->",entidad[1])
                    print(longitud,"longitud")
                    data.guardarTriplesGenerados(paper,frase,pos, entidad[0], relacion, entidad[1], 'resumen', longitud)
                    pos = pos + 1

 # Codigo reutilizado

In [ ]:
import re 
import string 
import nltk 

#from spacy.lang.en import English
import spacy
#import textacy


import pandas as pd 
import numpy as np 
import math 
from tqdm import tqdm 

from spacy.matcher import Matcher, DependencyMatcher
from spacy.tokens import Span 
from spacy import displacy

from collections import Counter, OrderedDict

In [ ]:
class AnalisisTexto:
    global nlp 
    nlp = spacy.load("en_core_web_sm")

In [ ]:
def get_entities(self,sent):
    ent1 = ""
    ent2 = ""

        prv_tok_dep = ""    # dependency tag of previous token in the sentence
        prv_tok_text = ""   # previous token in the sentence

        prefix = ""
        modifier = ""  
        for tok in nlp(sent): 
            if tok.dep_ != "punct":

            if tok.dep_ == "compound":
                prefix = tok.text

            if prv_tok_dep == "compound":
                prefix = prv_tok_text + " "+ tok.text
                
            if tok.dep_.endswith("mod") == True:
                modifier = tok.text

            if prv_tok_dep == "compound":
                modifier = prv_tok_text + " "+ tok.text
                
            if tok.dep_.find("subj") == True:
                ent1 = modifier +" "+ prefix + " "+ tok.text
            prefix = ""
            modifier = ""
            prv_tok_dep = ""
            prv_tok_text = "" 
                
            if tok.dep_.find("obj") == True:
                ent2 = modifier +" "+ prefix +" "+ tok.text       
            prv_tok_dep = tok.dep_
            prv_tok_text = tok.text  
        return [ent1.strip(), ent2.strip()]

In [ ]:
def get_relacion(self,sent):
    doc = nlp(sent)
    matcher = Matcher(nlp.vocab)
    pattern = [{'DEP':'ROOT'},
                {'DEP':'prep','OP':"?"},
                {'DEP':'agent','OP':"?"},
                {'POS':'ADJ','OP':"?"}]

        matcher.add('matching_1',[pattern])

        matches = matcher(doc)
        if len(matches):
            k = len(matches) - 1
            span = doc[matches[k][1]:matches[k][2]]
            return(span.text)
        else:
    return None